In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
from collections import defaultdict
from PIL import Image
%matplotlib inline

In [2]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Activation, AveragePooling2D, Cropping2D, Reshape, BatchNormalization
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.losses import mean_squared_error, binary_crossentropy
from keras.preprocessing.image import Iterator
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.engine.topology import Layer

import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [171]:
import sys
sys.path.append("/Users/l.chanussot/perso/carvana/")

In [172]:
from carvana.unet import get_model, preprocess

In [173]:
unet = get_model(512, 512, 3,
                     n_filters=[16, 32, 64, 128, 256])

In [175]:
unet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 3)   0                                            
____________________________________________________________________________________________________
conv1_1 (Conv2D)                 (None, 512, 512, 16)  448         input_1[0][0]                    
____________________________________________________________________________________________________
conv1_2 (Conv2D)                 (None, 512, 512, 16)  2320        conv1_1[0][0]                    
____________________________________________________________________________________________________
max_pool1 (MaxPooling2D)         (None, 256, 256, 16)  0           conv1_2[0][0]                    
___________________________________________________________________________________________

In [188]:
img1 = np.array(Image.open("../data/train_960x640/3cb21125f126_04.png"))
mask1 = np.array(Image.open("../data/train_masks_960x640/3cb21125f126_04_mask.png"))

In [177]:
pimg1 = preprocess(img1)

In [189]:
p = pimg1[100:100 + 512, 200:200 + 512, :]
m = mask1[100:100 + 512, 200:200 + 512]

In [182]:
p.shape

(512, 512, 3)

In [190]:
p = p.reshape((1, 512, 512, 3))
m = m.reshape((1, 512, 512, 1))

In [191]:
p.shape

(1, 512, 512, 3)

In [192]:
m.shape

(1, 512, 512, 1)

In [185]:
pp = unet.predict(p)

In [186]:
pp.shape

(1, 512, 512, 1)

In [193]:
y_true = K.constant(m)
y_pred = K.constant(pp)

In [194]:
with sess.as_default():
    print(dice_coef(y_true, y_pred, 1.0).eval())

0.507682


In [197]:
with sess.as_default():
    print(-np.log(dice_coef(y_true, y_pred, 1.0).eval()))

0.6779


In [165]:
y_true_np = np.ones((5, 512, 512, 1))
y_pred_np = np.ones((5, 512, 512, 1))
y_pred_npz = np.zeros((5, 512, 512, 1))
y_pred_npm = np.ones((5, 512, 512, 1))

In [166]:
y_true = K.constant(y_true_np)
y_pred = K.constant(y_pred_np)
y_predz = K.constant(y_pred_npz)
y_predm = K.constant(y_pred_npm)

In [167]:
def dice_coef(y_true, y_pred, smooth):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) /\
           (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [168]:
smooth = 1.0

In [169]:
with sess.as_default():
    print(-np.log(dice_coef(y_true, y_predm, 1.0).eval()))

-0.0


In [154]:
y_true_f = K.flatten(y_true)
y_pred_f = K.flatten(y_pred)
y_pred_fz = K.flatten(y_predz)

In [101]:
smooth = 1.0

In [102]:
a = (2. * intersection + smooth) /\
           (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [103]:
y_true_f

<tf.Tensor 'Reshape_29:0' shape=(1310720,) dtype=float32>

In [104]:
intersection = K.sum(y_true_f * y_pred_fz)

In [105]:
intersection

<tf.Tensor 'Sum_55:0' shape=() dtype=float32>

In [106]:
with sess.as_default():
    print((K.sum(y_true_f) + K.sum(y_pred_f) + smooth).eval())

2.62144e+06


In [119]:
-np.log(0.34)

1.0788096613719298

In [107]:
with sess.as_default():
    print(dice_coef(y_true, y_predz, 1.0).eval())

7.62939e-07


In [93]:
-1/np.log(1 - 0.997 + 1e-9)

0.17214244150095315

In [41]:
with sess.as_default():
    print(dice_coef(y_true, y_predz, 1e-6).eval())

3.33333e-09


In [12]:
loss_per_pixel = K.binary_crossentropy(y_pred, y_true_mask)

In [13]:
loss_per_pixel * y_true_weights

<tf.Tensor 'mul:0' shape=(5, 6, 10, 1) dtype=float32>

In [14]:
def weighted_binary_crossentropy(y_true, y_pred):
    y_true_weights = y_true[..., 1:2]
    y_true_mask = y_true[..., 0:1]
    loss_per_pixel = K.binary_crossentropy(y_pred, y_true_mask)
    return loss_per_pixel * y_true_weights

In [15]:
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_mask = y_true[..., 0:1]
    y_true_f = K.flatten(y_true_mask)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [16]:
def binary_accuracy(y_true, y_pred):
    y_true_mask = y_true[..., 0:1]
    return K.mean(K.equal(y_true_mask, K.round(y_pred)), axis=-1)

In [17]:
with sess.as_default():
    g = dice_coef(y_true, y_pred).eval()

In [28]:
y_true_np = np.ones((5, 6, 10, 1))
y_pred_np = np.ones((5, 6, 10, 1))
y_true_np[0, 1, 2, 0] = 0
y_true_np[4, 0, 1, 0] = 0

In [29]:
y_true = K.constant(y_true_np)
y_pred = K.constant(y_pred_np)

In [30]:
def class_weighted_binary_crossentropy(y_true, y_pred, weights):
    loss_per_pixel = K.binary_crossentropy(y_pred, y_true)
    weights_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]
    return loss_per_pixel * weights_per_pixel

In [31]:
weights = np.array([0.2, 0.8])

In [32]:
class_weighted_binary_crossentropy(y_true, y_pred, weights)

<tf.Tensor 'mul_10:0' shape=(5, 6, 10, 1) dtype=float32>

In [33]:
weights_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]

In [36]:
with sess.as_default():
    weights_per_pixel_e = weights_per_pixel.eval()

In [38]:
weights_per_pixel_e[4, 0, 1, 0]

0.2

In [39]:
def class_weighted_binary_accuracy(y_true, y_pred, weights):
    weight_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]
    accuracy_per_pixel = K.equal(y_true, K.round(y_pred))
    return K.mean(accuracy_per_pixel * weight_per_pixel, axis=-1)

In [40]:
weight_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]

In [41]:
weight_per_pixel

<tf.Tensor 'add_7:0' shape=(5, 6, 10, 1) dtype=float32>

In [42]:
accuracy_per_pixel = K.equal(y_true, K.round(y_pred))

In [46]:
K.cast(accuracy_per_pixel, K.floatx())

<tf.Tensor 'Cast:0' shape=(5, 6, 10, 1) dtype=float32>

In [44]:
accuracy_per_pixel * weight_per_pixel

ValueError: Tensor conversion requested dtype bool for Tensor with dtype float32: 'Tensor("add_7:0", shape=(5, 6, 10, 1), dtype=float32)'